In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/MyDrive/NLP


In [ ]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import import_ipynb

from nlp_Preproc_final import preproc
from nlp_model_final import get_model, get_model_with_params, BertModelInitialization
import random
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import time

importing Jupyter notebook from nlp_Preproc_final.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
importing Jupyter notebook from nlp_model_final.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 정확도 계산 함수
def accuracy(preds, labels):
    f_pred = np.argmax(preds, axis=1).flatten()
    f_labels = labels.flatten()
    return np.sum(f_pred == f_labels) / len(f_labels)

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 2022

**잠깐 ✔ 랜덤시드 고정이란 무엇인가?**
> 학습된 모델의 결과를 동일하게 재현(Reproduction)하는 것은 여러가지 상황에서 팔요하다.  
> 모델을 돌릴 때마다 결과가 달라지지 않도록 고정하는 것이다.

- 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

- 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

- 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등 여러 상황에서 필요하다.

참고 자료:
https://dacon.io/codeshare/2363
https://pytorch.org/docs/stable/notes/randomness.html


In [ ]:
# 랜덤하게 데이터를 추출하기 위한 seed 값 설정
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
from nlp_tokenization import KoBertTokenizer

# 전체 데이터를 불러오자.
whole_dataset = pd.read_excel('/content/gdrive/MyDrive/NLP/data/chat_data.xlsx')

# KoBERTTokenizer를 불러오자.
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

train_dataloader, validation_dataloader = preproc(tokenizer, whole_dataset)

length of whole_data : 34388


nlp_Preproc_final.ipynb:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [ ]:
# BertModel 생성해서 GPU 혹은 CPU에 등록
# 기존 Device에 등록된 BertModel은 초기화되니, 유의하여 사용할 것.
# 한 번만 실행하고, 그 이후로는 사용하지 않도록 조심!
BertModelInitialization()

In [ ]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, optimizer, scheduler, epochs, criterion = get_model_with_params(len(train_dataloader), device, torch.cuda.is_available())

# 그래디언트 초기화
model.zero_grad()

In [ ]:
# 에폭만큼 반복
for epoch_i in range(epochs):
    print("")
    print('========{:}번째 Epoch / 전체 {:}회 ========'.format(epoch_i + 1, epochs))
    print('훈련 중')

    t0 = time.time() # 시작 시간 설정
    total_loss = 0 # 로스 초기화
    sum_loss = 0
    model.train()  # 훈련모드로 변경
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader)):
        
        if step % 50 == 0:
          print("{}번째 까지의 평균 loss : {}".format(step, sum_loss/50))
          sum_loss = 0

        batch = tuple(t.to(device) for t in batch)   # 배치를 GPU에 넣음
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 데이터 추출

        # nlp_model_final.ipynb 파일에서 BertClassifier 클래스를 정의하고 해당 클래스를 활용하여 'model'을 생성한 바 있다.
        # 위의 코드에서 정의한 배치 데이터를 'model'의 입력 변수로 이용하여, 배치에 대한 forward를 수행해보자.   
        # Hint: BertClassifier의 forward를 수행하기 위해서는 input_ids, attention_mask 변수를 입력받아야 한다. 
        ## 여기에 코드 작성
        b_input_ids, b_input_mask = torch.Tensor(b_input_ids), torch.Tensor(b_input_mask)
        outputs = model.forward(input_ids = b_input_ids, attention_mask = b_input_mask)


        # 위에서 산출된 7차원 벡터와 criterion, b_labels를 활용해서 loss를 계산해보자.
        ## 여기에 코드 작성
        loss = criterion(outputs, b_labels)

        total_loss += loss.item() # 총 로스 계산
        sum_loss += loss.item()

        loss.backward() # Backward 수행으로 그래디언트 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑
        optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
        scheduler.step()  # 스케줄러로 학습률 감소
        model.zero_grad() # 그래디언트 초기화

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    
    #### 검증 ####
    
    print("")
    print("검증 중")

    t0 = time.time() #시작 시간
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():
            # 위 train과 같은 논리. forward를 수행해서 7차원 벡터 출력    
            # outputs = ~~ 과 같이 작성
            ## 여기에 코드 작성
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
        # CPU로 데이터 이동
        outputs = outputs.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 위에 구현한 accuracy 함수를 활용해서 출력 로짓과 라벨을 비교하여 정확도 계산
        # tmp_eval_accuracy = ~~ 와 같이 작성
        ## 여기에 코드 작성
        tmp_eval_accuracy = accuracy(outputs, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))

# 학습된 모델을 해당 PATH에 저장
PATH = "model_5.pt"
torch.save(model.state_dict(), PATH)

print("")
print("Training complete!")


========1번째 Epoch / 전체 10회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 1.956652889251709
100번째 까지의 평균 loss : 1.9659754252433777
150번째 까지의 평균 loss : 1.9510446906089782
200번째 까지의 평균 loss : 1.9376977038383485
250번째 까지의 평균 loss : 1.9237706446647644
300번째 까지의 평균 loss : 1.899645299911499
350번째 까지의 평균 loss : 1.8669838142395019
400번째 까지의 평균 loss : 1.8193866300582886
450번째 까지의 평균 loss : 1.7602836322784423
500번째 까지의 평균 loss : 1.6758666110038758
550번째 까지의 평균 loss : 1.631530523300171
600번째 까지의 평균 loss : 1.551761360168457
650번째 까지의 평균 loss : 1.5327373600006104
700번째 까지의 평균 loss : 1.4770533561706543
750번째 까지의 평균 loss : 1.4501932907104491
800번째 까지의 평균 loss : 1.4332912683486938
850번째 까지의 평균 loss : 1.4169037652015686
900번째 까지의 평균 loss : 1.4026554584503175
950번째 까지의 평균 loss : 1.3734631109237672

  Average training loss: 1.68

검증 중
  Accuracy: 0.4984

========2번째 Epoch / 전체 10회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 1.3292883956432342
100번째 까지의 평균 loss : 1.336618300676346
150번째 까지의 평균 loss : 1.2759038138389587
200번째 까지의 평균 loss : 1.2793341982364654
250번째 까지의 평균 loss : 1.2473099052906036
300번째 까지의 평균 loss : 1.3233742582798005
350번째 까지의 평균 loss : 1.2580222439765931
400번째 까지의 평균 loss : 1.2356862032413483
450번째 까지의 평균 loss : 1.222420471906662
500번째 까지의 평균 loss : 1.211978211402893
550번째 까지의 평균 loss : 1.2471224451065064
600번째 까지의 평균 loss : 1.2377900314331054
650번째 까지의 평균 loss : 1.2744945144653321
700번째 까지의 평균 loss : 1.1859416258335114
750번째 까지의 평균 loss : 1.2139047467708588
800번째 까지의 평균 loss : 1.1556897366046905
850번째 까지의 평균 loss : 1.1969659316539765
900번째 까지의 평균 loss : 1.1776805877685548
950번째 까지의 평균 loss : 1.240157824754715

  Average training loss: 1.24

검증 중
  Accuracy: 0.5381

========3번째 Epoch / 전체 10회 ========
훈련 중


...

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.6169940704107284
100번째 까지의 평균 loss : 0.6311992824077606
150번째 까지의 평균 loss : 0.6072546064853668
200번째 까지의 평균 loss : 0.6280286687612534
250번째 까지의 평균 loss : 0.6160051292181015
300번째 까지의 평균 loss : 0.6212901782989502
350번째 까지의 평균 loss : 0.633393462896347
400번째 까지의 평균 loss : 0.6601703631877899
450번째 까지의 평균 loss : 0.6461535823345185
500번째 까지의 평균 loss : 0.6269835966825485
550번째 까지의 평균 loss : 0.639195756316185
600번째 까지의 평균 loss : 0.6381707525253296
650번째 까지의 평균 loss : 0.6224606055021286
700번째 까지의 평균 loss : 0.6390034711360931
750번째 까지의 평균 loss : 0.6577186608314514
800번째 까지의 평균 loss : 0.6063681173324585
850번째 까지의 평균 loss : 0.596800297498703
900번째 까지의 평균 loss : 0.6163455885648728
950번째 까지의 평균 loss : 0.6301133334636688

  Average training loss: 0.63

검증 중
  Accuracy: 0.5484

========9번째 Epoch / 전체 10회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.6056139647960663
100번째 까지의 평균 loss : 0.5616042506694794
150번째 까지의 평균 loss : 0.5888723260164261
200번째 까지의 평균 loss : 0.580210400223732
250번째 까지의 평균 loss : 0.5441458788514137
300번째 까지의 평균 loss : 0.5919910657405854
350번째 까지의 평균 loss : 0.5827600568532944
400번째 까지의 평균 loss : 0.6100363123416901
450번째 까지의 평균 loss : 0.5768812185525894
500번째 까지의 평균 loss : 0.582354592680931
550번째 까지의 평균 loss : 0.5493140888214111
600번째 까지의 평균 loss : 0.574761130809784
650번째 까지의 평균 loss : 0.559155045747757
700번째 까지의 평균 loss : 0.5725301375985146
750번째 까지의 평균 loss : 0.5762936693429946
800번째 까지의 평균 loss : 0.5939389717578888
850번째 까지의 평균 loss : 0.5986828362941742
900번째 까지의 평균 loss : 0.5794332778453827
950번째 까지의 평균 loss : 0.6107359057664872

  Average training loss: 0.58

검증 중
  Accuracy: 0.5471

========10번째 Epoch / 전체 10회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.5314936167001725
100번째 까지의 평균 loss : 0.5796299135684967
150번째 까지의 평균 loss : 0.5253195071220398
200번째 까지의 평균 loss : 0.5617330551147461
250번째 까지의 평균 loss : 0.5625067353248596
300번째 까지의 평균 loss : 0.6062152916193009
350번째 까지의 평균 loss : 0.5510756257176399
400번째 까지의 평균 loss : 0.5392329072952271
450번째 까지의 평균 loss : 0.5426161548495293
500번째 까지의 평균 loss : 0.5878934460878372
550번째 까지의 평균 loss : 0.5406257295608521
600번째 까지의 평균 loss : 0.5632197046279908
650번째 까지의 평균 loss : 0.5122175139188766
700번째 까지의 평균 loss : 0.5708084261417389
750번째 까지의 평균 loss : 0.5580396229028701
800번째 까지의 평균 loss : 0.5578826785087585
850번째 까지의 평균 loss : 0.5320330715179443
900번째 까지의 평균 loss : 0.5524066108465194
950번째 까지의 평균 loss : 0.538345907330513

  Average training loss: 0.55

검증 중
  Accuracy: 0.5463

Training complete!
